# Time Series Forecasting with Transformer in PyTorch
This notebook demonstrates how to use a simple Transformer architecture for time series forecasting using PyTorch.

In [ ]:
# Import required libraries
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import TensorDataset, DataLoader
from data_utils import generate_sine_wave
from transformer_model import TimeSeriesTransformer

In [ ]:
# Hyperparameters
SEQ_LENGTH = 20
NUM_SAMPLES = 1000
BATCH_SIZE = 32
EPOCHS = 20
LR = 0.001

In [ ]:
# Generate data
X, y = generate_sine_wave(SEQ_LENGTH, NUM_SAMPLES)
X_train, y_train = X[:800], y[:800]
X_test, y_test = X[800:], y[800:]

In [ ]:
# Convert to tensors and create DataLoader
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

In [ ]:
# Model, optimizer, and loss function
model = TimeSeriesTransformer(input_size=1).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=LR)
criterion = nn.MSELoss()

In [ ]:
# Training loop using DataLoader
for epoch in range(EPOCHS):
    model.train()
    epoch_loss = 0
    for batch_x, batch_y in train_loader:
        batch_x = batch_x.to(device)
        batch_y = batch_y.to(device)
        optimizer.zero_grad()
        output = model(batch_x)
        loss = criterion(output, batch_y)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item() * batch_x.size(0)
    avg_loss = epoch_loss / len(train_loader.dataset)
    print(f"Epoch {epoch+1}/{EPOCHS}, Loss: {avg_loss:.4f}")

In [ ]:
# Evaluation and prediction using DataLoader
model.eval()
preds = []
true_vals = []
with torch.no_grad():
    for batch_x, batch_y in test_loader:
        batch_x = batch_x.to(device)
        batch_y = batch_y.to(device)
        output = model(batch_x)
        preds.append(output.cpu().numpy())
        true_vals.append(batch_y.cpu().numpy())
preds = np.concatenate(preds)
true_vals = np.concatenate(true_vals)
test_loss = criterion(torch.tensor(preds), torch.tensor(true_vals)).item()
print(f"Test Loss: {test_loss:.4f}")

In [ ]:
# Plot predictions
plt.figure(figsize=(10,4))
plt.plot(true_vals, label='True')
plt.plot(preds, label='Predicted')
plt.legend()
plt.title('Time Series Forecasting with Transformer')
plt.show()